# **EXPLORER FOR SEC FILINGS**
<hr>

## Inter IIT Tech Meet 10.0 (2022)

![image](https://www.sec.gov/edgar/search/images/edgar-logo-2x.png)
![image](https://interiit-tech.org/static/media/logo_1.f4d40e83.png)

In this Notebook, we shall be looking into utilizing the [EDGAR](https://www.sec.gov/edgar/searchedgar/) API to explore the SEC filings of a company. We shall be using the python library python-edgar to access the API. Be careful, the API is limited to 10 requests per second or smth, idk. If a black SUV shows up out in the open, it's probably because you're doing something wrong.

## Objectives
- Scrape Data from the company's History since inception
- Use 10-Q 10-K and 8-K filings to get the company's financial statements
- Use Financial Statements to get the company's balance sheet, income statement, cash flow statement, and ratios
- Use the data to get the company's current assets, liabilities, and equity
- Generate SaaS Metrics
- Generate a Financial Statement Analysis
- Use the metrics with Deep Learning Systems to give Insightful Results

Graciaz<br>
Kaushik Dey

In [37]:
import requests as req
import pandas as pd
import xml.etree.ElementTree as et
import matplotlib.pyplot as plt
import pandasgui as pdgui
import os
import numpy as np
import json
import time
import pyjsonviewer as pjv
from bs4 import BeautifulSoup as bs

## Parsing 10-K Documents

#### What is a 10-K Form?
A Form 10-K is an annual report required by the U.S. Securities and Exchange Commission (SEC), that gives a comprehensive summary of a company's financial performance. Some of the information a company is required to document in the 10-K includes its history, organizational structure, financial statements, earnings per share, subsidiaries, executive compensation, and any other relevant data.


The SEC requires this report to keep investors aware of a company's financial condition and to allow them to have enough information before they buy or sell shares in the corporation, or before investing in the firm’s corporate bonds.

#### Why we need 10-K Forms?
The SEC mandates that all public companies file regular 10-Ks to keep investors aware of a company's financial condition and to allow them to have enough information before they buy or sell securities issued by that company. The 10-K can appear overly complex at first glance, complete with tables full of data and figures. However, it is because it is so comprehensive that this filing is key for investors to get a handle on a company's financial position and prospects.

The Form 10-K is comprised of several parts. These include:

- **Business summary:** This describes the company's operations. It would include information about business segments, products and services, subsidiaries, markets, regulatory issues, research and development, competition, and employees, among other details.
- **Management Discussion and Analysis:** This section allows the company to explain its operations and financial results for the past year.
- **Financial statements:** The financial statements would include the company's balance sheet, income statement, and cash flow statement.
- **Additional sections:** Additional sections may discuss the company's management team and legal proceedings.


In [38]:
file_base = "https://sec.gov/Archives/edgar/data/746210/000074621021000024/" # Base URL for filing
filing_summary = file_base + "FilingSummary.xml"

head = {
    "User-Agent": "Alpha-Explorer/1.0",
    "Connection": "keep-alive"
}

res = req.get(filing_summary, headers=head)

root = et.fromstring(res.text)
root.tag

'FilingSummary'

Filing Summary is essential as it will help us show the components of the 10-K in tabulated HTM

In [39]:
#Exclude last entry or it creates an error
component_dict = [] # Array to store our dictionary
trigger_list = ['BALANCE SHEET', 'INCOME', 'CASH FLOW', 'EQUITY'] # List of triggers to look for
for report in root.iter('Report'):
    dict = {}
    for trigger_word in trigger_list:
        if trigger_word in report.find('ShortName').text:
            try:
                dict["name"] = report.find('ShortName').text
                dict["url"] = file_base+report.find('HtmlFileName').text
                component_dict.append(dict)
            except:
                print("No report found at some point")

component_dict

[{'name': 'CONSOLIDATED BALANCE SHEETS',
  'url': 'https://sec.gov/Archives/edgar/data/746210/000074621021000024/R2.htm'},
 {'name': 'CONSOLIDATED BALANCE SHEETS (Parenthetical)',
  'url': 'https://sec.gov/Archives/edgar/data/746210/000074621021000024/R3.htm'},
 {'name': "CONSOLIDATED STATEMENT OF STOCKHOLDERS' EQUITY",
  'url': 'https://sec.gov/Archives/edgar/data/746210/000074621021000024/R5.htm'},
 {'name': 'CONSOLIDATED STATEMENTS OF CASH FLOWS',
  'url': 'https://sec.gov/Archives/edgar/data/746210/000074621021000024/R6.htm'},
 {'name': 'CONSOLIDATED STATEMENTS OF CASH FLOWS (Parenthetical)',
  'url': 'https://sec.gov/Archives/edgar/data/746210/000074621021000024/R7.htm'}]

### Extracting Tables into a Dictionary
- The tables in the 10-K are in a html format
- Create a dict where every data is stored categorized in headers and sections with data

```html
<table class="report" border="0" cellspacing="2" id="idm139636460643688">
    <tr>
        <th class="tl" colspan="1" rowspan="1"><div style="width: 200px;">
            <strong>CONSOLIDATED BALANCE SHEETS - USD ($)<br> $ in Thousands</strong></div>
        </th>
        <th class="th">
            <div>Dec. 31, 2020</div>
        </th>
        <th class="th">
            <div>Dec. 31, 2019</div>
        </th>
    </tr>
    <tr class="re">
        <td class="pl " style="border-bottom: 0px;" valign="top">
            <a class="a" href="javascript:void(0);"><strong>Current assets:</strong></a>
        </td>
        <td class="text">
            &#160;<span></span>
        </td>
        <td class="text">
            &#160;<span></span>
        </td>
    </tr>
    <tr class="ro">
        <td class="pl " style="border-bottom: 0px;" valign="top">
            <a class="a" href="javascript:void(0);">Cash</a>
        </td>
        <td class="nump">
            $ 5,058<span></span>
        </td>
        <td class="nump">
            $ 4,602<span></span>
        </td>
    </tr>
```

Some details we can see here are:

- header rows are ez to recognize, they have `<th>` tags
- sections are however, not so ez, they have `<strong>` tags inside, but the tags are `<td>`, also the empty ones have `&#160;<span></span>` within them
- the numbers are in the `<td>` tags, but they are in `<span>` tags, so we need to extract the numbers.

In [40]:
head = {
    "User-Agent": "Alpha-Explorer/1.0",
    "Connection": "keep-alive"
}

keypoints = [
    'profit',
    'total',
    'beginning balance',
    'net loss',
    'ending balance',
    'stock',
    'net cash',
    
]

def has_keypoints(str, list):
    for key in list:
        if key in str.lower():
            return True
    return False

for component in component_dict:
    res = req.get(component["url"], headers=head)
    soup = bs(res.text, "html")

    #Sample Data

    table = {}

    table['headers'] = []
    table['rows'] = []
    table['sections'] = []

    for i, row in enumerate(soup.table.find_all('tr')):

        # Data Row
        if (len(row.find_all('th')) == 0 and len(row.find_all('strong')) == 0):
            dat = [ e.text.strip() for e in row.find_all('td') ]
            if has_keypoints(dat[0],keypoints): table['rows'].append(dat)
        #Section Row
        elif (len(row.find_all('th')) == 0 and len(row.find_all('strong')) != 0):
            dat = [ e.text.strip() for e in row.find_all('strong') ]
            table['sections'].append(dat)
        #Header Row
        elif len(row.find_all('th')) != 0:
            dat = [ e.text.strip() for e in row.find_all('th') ]
            table['headers'].append(dat)
        else:
            print('Error @ Row ', i, ' In Table ', dat['name'])
    
    # Convert to DF and remove all unneccessary characters
    try:
        component['table'] = pd.DataFrame(table['rows']).replace('[\$,)]','', regex=True ).replace( '[(]','-', regex=True).replace( '', 'NaN', regex=True)
        component['table'].columns = table['headers']
    except:
        print("No data found at ", component['url'])

    table.clear()

No data found at  https://sec.gov/Archives/edgar/data/746210/000074621021000024/R6.htm
No data found at  https://sec.gov/Archives/edgar/data/746210/000074621021000024/R7.htm


### Arranging Data into DF and providing headers

In [55]:
component_dict[3]['table']

,0,1,2
0,Net loss,-6341,-8598
1,Stock-based compensation,535,89
2,Stock-based expense for services,390,0
3,Net cash used in operating activities,-5956,-4325
4,Net cash used in investing activities,-30,-8
5,Proceeds from stock issuance net of issuance c...,11504,0
6,Purchase of treasury stock,0,-16
7,Net cash provided by financing activities,11504,2401
8,Total cash and restricted cash,10795,2670
9,Issuance of preferred stock in exchange for ac...,0,99


### Now we shall proceed to process a 10-Q form

- Fetch a 10-Q form from the SEC
- Analyse tables
- Extract data
- Create a DataFrame
- Sheeeeesh

In [47]:
file_base = "https://www.sec.gov/Archives/edgar/data/746210/000074621021000062/" # Base URL for filing
filing_summary = file_base + "FilingSummary.xml"

head = {
    "User-Agent": "Alpha-Explorer/1.0",
    "Connection": "keep-alive"
}

res = req.get(filing_summary, headers=head)

root = et.fromstring(res.text)
root.tag

'FilingSummary'

In [48]:
#Exclude last entry or it creates an error
component_dict = [] # Array to store our dictionary
trigger_list = ['BALANCE SHEET', 'INCOME', 'CASH FLOW', 'EQUITY'] # List of triggers to look for
for report in root.iter('Report'):
    dict = {}
    for trigger_word in trigger_list:
        if trigger_word in report.find('ShortName').text:
            try:
                dict["name"] = report.find('ShortName').text
                dict["url"] = file_base+report.find('HtmlFileName').text
                component_dict.append(dict)
            except:
                print("No report found at some point")

component_dict

[{'name': 'CONDENSED CONSOLIDATED BALANCE SHEETS',
  'url': 'https://www.sec.gov/Archives/edgar/data/746210/000074621021000062/R2.htm'},
 {'name': 'CONDENSED CONSOLIDATED BALANCE SHEETS (Parenthetical)',
  'url': 'https://www.sec.gov/Archives/edgar/data/746210/000074621021000062/R3.htm'},
 {'name': "CONDENSED CONSOLIDATED STATEMENT OF STOCKHOLDERS' EQUITY",
  'url': 'https://www.sec.gov/Archives/edgar/data/746210/000074621021000062/R5.htm'},
 {'name': 'CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS',
  'url': 'https://www.sec.gov/Archives/edgar/data/746210/000074621021000062/R6.htm'}]

In [49]:
head = {
    "User-Agent": "Alpha-Explorer/1.0",
    "Connection": "keep-alive"
}

keypoints = [
    'profit',
    'total',
    'beginning balance',
    'net loss',
    'ending balance',
    'stock',
    'net cash',
    
]

def has_keypoints(str, list):
    for key in list:
        if key in str.lower():
            return True
    return False

for component in component_dict:
    res = req.get(component["url"], headers=head)
    soup = bs(res.text, "html")

    #Sample Data

    table = {}

    table['headers'] = []
    table['rows'] = []
    table['sections'] = []

    for i, row in enumerate(soup.table.find_all('tr')):

        # Data Row
        if (len(row.find_all('th')) == 0 and len(row.find_all('strong')) == 0):
            dat = [ e.text.strip() for e in row.find_all('td') ]
            if has_keypoints(dat[0],keypoints): table['rows'].append(dat)
        #Section Row
        elif (len(row.find_all('th')) == 0 and len(row.find_all('strong')) != 0):
            dat = [ e.text.strip() for e in row.find_all('strong') ]
            table['sections'].append(dat)
        #Header Row
        elif len(row.find_all('th')) != 0:
            dat = [ e.text.strip() for e in row.find_all('th') ]
            table['headers'].append(dat)
        else:
            print('Error @ Row ', i, ' In Table ', dat['name'])
    
    # Convert to DF and remove all unneccessary characters
    try:
        component['table'] = pd.DataFrame(table['rows']).replace('[\$,)]','', regex=True ).replace( '[(]','-', regex=True).replace( '', 'NaN', regex=True)
        component['table'].columns = table['headers']
    except:
        print("No data found at ", component['url'])

    table.clear()

No data found at  https://www.sec.gov/Archives/edgar/data/746210/000074621021000062/R6.htm


In [51]:
component_dict[1]['table']

,CONDENSED CONSOLIDATED BALANCE SHEETS (Parenthetical) - USD ($),"Sep. 30, 2021","Dec. 31, 2020"
0,Preferred stock convertible shares authorized ...,5000000,NaN
1,Preferred stock convertible shares issued -in ...,0,NaN
2,Preferred stock convertible shares outstanding...,0,1829582
3,Common stock par value -in dollars per share,0.0001,0.0001
4,Common stock shares authorized -in shares,150000000,150000000
5,Common stock shares issued -in shares,30929331,7861912
6,Common stock shares outstanding -in shares,30816048,7748629
7,Treasury stock shares -in shares,113283,113283
8,Series A-2 Preferred Stock,NaN,NaN
9,Preferred stock convertible par value -in doll...,0.0001,0.0001
